In [2]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import momepy
import networkx as nx

In [3]:
%load_ext autoreload
%autoreload 2

##### STEP 1 - READ NETWORK AND SIMULATION DATA
Read network and simulation data from D-Hydro simulations using his.nc and map.nc:
- for sets WINTER and SUMMER and ...
- for list of SIMULATIONS per set (in this case 1 simulation per set)
- at list of TIMESTEPS per simulation (in this case multiple timesteps per simulation)

and read network

In [10]:
base_dir = Path("c:\\Users\\NLHARN\\Documents\\SWECO_LOKAAL\\NHI-TKI-oppervlaktewatermodule-Ribasim\\TKI-NHI-oppervlaktewatermodule\\")
# base_dir = Path("C:\\Users\\NLTAND\\OneDrive - Sweco AB\\Algemeen-Tessa\\Projecten\\TKI oppervlaktewatermodule NHI\\")
simulations_dir = Path(base_dir, "d-hydro\\dhydro\\")
date_range = pd.date_range("2000-01-02 23:00", periods=9, freq="2D")

# load areas units (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "wrij_afwateringseenheden_clip_Zutphen\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas = gpd.read_file(areas_file_path)[["geometry"]].to_crs(28992)

# output geopackage
gpkg_path = "output/test_export_geopackage/ribasim_network.gpkg"

In [11]:
# create RibasimLumpingNetwork
network = RibasimLumpingNetwork(areas_gdf=areas)

# add data from simulation sets
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)

# get all network data from the schematisations
network.get_network_data()

Simulation set (winter): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc
Simulation set (zomer): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc
network locations read: nodes/edges/confluences/bifurcations/weirs/pumps/laterals


##### STEP 2: SPLIT NETWORK IN RIBASIM BASINS
2. Split network in ribasim basins  
    1. Specify model cut locations (id nodes or xy coordinates)  
    2. Dividing the network based on the clipping locations (function to be built in xugrid?)  
        1. Convert to networkx graph
        2. Split network
        3. TODO: What to do if you want to distinguish areas that drain into a main watercourse as a basin with the end of the inflow point? Have choices made between:</dd>
            - simple: also cut the main waterway at this junction or
            - difficult: continue to see the parts upstream and downstream of the inflow point together as one basin
    3. Based on the split network, associated drain areas are determined by means of spatial join.
    4. Use representative locations of areas as node location.

In [29]:
# Define node_ids on which to split
# - define types to include
split_node_ids_1 = network.get_node_ids_from_type(
    bifurcations=False,
    confluences=True,
    weirs=True,
    laterals=False
)
# - define by node code
split_node_ids_2 = [1452, 2372]
# - define which of the node_ids should be excluded
split_node_ids_exclude = [314]

# - combine types of split_node_ids
split_node_ids = [node_id for node_id in split_node_ids_1 + split_node_ids_2 
                  if node_id not in split_node_ids_exclude]

In [30]:
# create basins (gdf) based on nodes, edges, split_node_ids and areas
results = network.create_basins_based_on_split_node_ids(split_node_ids=split_node_ids)
# export it to geopackage
network.export_to_geopackage(output=gpkg_path)

In [31]:
# adaptations to network
#  - TODO: define which basins to combine

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\NLHARN\Anaconda3\envs\ribasim\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\NLHARN\Anaconda3\envs\ribasim\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] De externe host heeft een verbinding verbroken
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\NLHARN\Anaconda3\envs\ribasim\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\NLHARN\Anaconda3\envs\ribasim\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] De 

##### STEP 3: CREATE SETTINGS RIBASIM BASINS
3. Create settings ribasim basins
    1. Determine volume-water level relations for the Basins. Relationship between total volume and water levels within basin and water level representative outflow point:
        1. Include all disposal in area: storage in calculation nodes and possible disposal nodes.
        2. Also include storage in tertiary system? Modflow or possibly RR?
        3. Practical point we ran into: What to do with the storage in the locations where you are just making the cut? Distribute evenly over the connected basins?
    2. Lump and add lateral inflow/outflow within areas including precipitation/evaporation.
        1. MODFLOW/MetaSWAP: Lump fluxes over area.
        2. D-Hydro: Lump inflow of laterals.


In [ ]:
# TODO: CHECK WITH HKV WHAT THEY ARE DOING AND MAYBE IMPORT THEIR FUNCTIONS

##### STEP 4: BOUNDARIES AND BOUNDARY CONDITIONS FOR INFLOW/OUTFLOW POINTS
4. Boundaries and boundary conditions of inflow/outflow points of the system/network
     1. Automatically identify network edges based on network and areas.
     2. Selecting borders and border type

##### STEP 5: CONNECTIONS BETWEEN RIBASIM - BASINS:
5. Connections Between Ribasim-Basins:
    1. Automatically generate connections between Basins. Determine per simulation set the flow and flow direction (winter/summer/supply):
        1. Specify preferred exchange type (maybe per split location or basin)
        2. Determine presence and direction based on flow direction.
        3. Place connector nodes near the cut locations (but not all on top of each other).
    2. Definition of exchange relations based on preference and manual choice model

##### STEP 6: GENERATE SETTINGS FOR EACH CONNECTION:
6. Generate settings for the exchanges:
    1. TabulatedRatingCurve: QH/QV relationships
    2. Manning: Manning and distance
    3. FractionalFlows for splits with distributions
    4. PID controller: Translate control from D-Hydro-RTC

In [ ]:
# TODO: CHECK WITH HKV WHAT THEY ARE DOING AND MAYBE IMPORT THEIR FUNCTIONS

##### STEP 7: ADD ALL TIME-DEPENDENT INPUT
7. Add all time-dependent inputs:
    1. Hydraulic boundary conditions.
    2. Precipitation on and evaporation from open water
    3. Lateral inflow/outflow to groundwater or tertiary system
    4. Surface water abstractions

In [ ]:
# TODO: CHECK WITH HKV WHAT THEY ARE DOING AND MAYBE IMPORT THEIR FUNCTIONS